## Code Hist.

 - CODE : KIER Data_단순 분리 (전처리 X)  
 - DESC  
    &ensp; : int_Domain 코드의 변경에 따라,   
    &emsp; KIER 에너지 사용량 데이터를 동/층/세대별로 분리 (10분 주기)   
    &emsp; 추가적으로, Raw Data과 세대별 데이터 추출 단계에서, 아래와 같은 작업 진행  
    &emsp;&emsp; 1) Date Column에 대한 유효성 검사 및 이상 Data에 대한 소거  
    &emsp;&emsp; 1-1) Datetime 유효성 확인  
    &emsp;&emsp; 1-2) Datetime 이상치 제거  
    &emsp;&emsp; 2) HOUSE_ID_HO의 해시값을 int_house_num으로 변경 (용량 절감 목적)  
  - DATE  
    &ensp; 2023-10-12 Created  
    &ensp; 2023-11-21 Code 최신화  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 기존 모듈화된 함수 사용  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 기존 KIER 변환 관련 코드를 모두 통합  
    &ensp; 2024-01-19 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) HOT (온수 사용량) 전용 Code 작성  
    &ensp; 2024-03-13 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 적산 및 순시사용량 산출 코드 추가  
    &ensp; 2024-07-24 Error Fix : 호실별 사용량 취합시 MemoryError 발생 방지  
    &ensp; 2024-07-25 Code 개선 : HOUSE_ID Feature 도입에 따른 Code 변경  
    &ensp; 2024-07-26 Code 간략화 : ACCU/INST 관련 Src 부분을 각각의 파일로 분리  

## Code

### 01. Init

#### 01-01. Module Import

In [81]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import tqdm
from tqdm.notebook import tqdm
#endregion Import_Basic Module

In [82]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch, torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf, tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [83]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8514041830015912304
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22395486208
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4148974483685089151
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
 xla_global_id: 416903419]

In [84]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    print('Using GPU')
    tf.config.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)
else : 
  print('Using CPU')

Using GPU


In [85]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02

#### 01-02. Config (Directory, Params)

In [86]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [87]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-07-30 02:22:37.726721
2024 / 7 / 30
2 : 22


### 02. Data Prepare

In [88]:
## Dict_Domain
int_domain = 5

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)

## File
str_fileRaw, str_fileRaw_hList = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv'), str('KIER_hList_Common.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

5 : GAS
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_GAS_561-1-1_ACCU_01_Raw.csv', 'KIER_GAS_561-1-1_INST_01_10min.csv', 'KIER_GAS_561-1-2_ACCU_01_Raw.csv', 'KIER_GAS_561-1-2_INST_01_10min.csv', 'KIER_GAS_561-1-3_ACCU_01_Raw.csv', 'KIER_GAS_561-1-3_INST_01_10min.csv', 'KIER_GAS_561-1-4_ACCU_01_Raw.csv', 'KIER_GAS_561-1-4_INST_01_10min.csv', 'KIER_GAS_561-10-1_ACCU_01_Raw.csv', 'KIER_GAS_561-10-1_INST_01_10min.csv', 'KIER_GAS_561-10-2_ACCU_01_Raw.csv', 'KIE

#### 02-01. KIER (hList)

In [89]:
## Load Dataset (hList)
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
# print(df_kier_hList.shape, ' /// ', df_kier_hList.columns)
# df_kier_hList

#### 02-02. KIER (Energy Usage)

##### Case 01. "METER_DATE" Cleansing이 완료되지 않은 경우

In [90]:
## ▶ Load Dataset (Energy Usage)
df_raw = pd.read_csv(str_dir_raw + str_fileRaw, index_col = 0)
print(df_raw.shape, ' /// ', df_raw.columns)
df_raw

(52379163, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'GAS_ACCU_FLOW'], dtype='object')


,METER_DATE,HOUSE_ID,GAS_ACCU_FLOW
823,2022-08-10 15:30:09,561-1-1,0.0000000000
824,2022-08-24 10:00:26,561-1-1,0.0000000000
825,2022-09-19 23:54:11,561-1-1,0.1200000000
826,2022-09-20 19:14:12,561-1-1,0.1200000000
827,2022-09-20 20:14:12,561-1-1,0.1200000000
...,...,...,...
52380804,2024-06-24 09:36:59,563-24-2,0.1100000000
52380805,2024-06-24 09:46:59,563-24-2,0.1100000000
52380806,2024-06-24 09:56:59,563-24-2,0.1100000000
52380807,2024-06-24 10:06:59,563-24-2,0.1100000000


In [91]:
## 1) Date Column에 대한 유효성 검사 및 이상 Data에 대한 소거
## 1-1) Datetime 유효성 확인
list_errValues = com_date.list_invalidDate(df_raw, 'METER_DATE')
print(len(list_errValues))

## 1-2) Datetime 이상치 제거
for i in range(len(list_errValues) - 1, -1, -1) : df_raw = df_raw.drop(index = list_errValues[i], axis = 0)

print(df_raw.shape, ' /// ', df_raw.columns)
df_raw

(52379163, 3)
0
[]
0
(52379163, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'GAS_ACCU_FLOW'], dtype='object')


,METER_DATE,HOUSE_ID,GAS_ACCU_FLOW
823,2022-08-10 15:30:09,561-1-1,0.0000000000
824,2022-08-24 10:00:26,561-1-1,0.0000000000
825,2022-09-19 23:54:11,561-1-1,0.1200000000
826,2022-09-20 19:14:12,561-1-1,0.1200000000
827,2022-09-20 20:14:12,561-1-1,0.1200000000
...,...,...,...
52380804,2024-06-24 09:36:59,563-24-2,0.1100000000
52380805,2024-06-24 09:46:59,563-24-2,0.1100000000
52380806,2024-06-24 09:56:59,563-24-2,0.1100000000
52380807,2024-06-24 10:06:59,563-24-2,0.1100000000


In [92]:
## Cleansed Data를 저장
str_file = str('KIER_RAW_' + str_domain + '_Cleansed.csv')
df_raw.to_csv(str_dir_raw + str_file)
df_raw

,METER_DATE,HOUSE_ID,GAS_ACCU_FLOW
823,2022-08-10 15:30:09,561-1-1,0.0000000000
824,2022-08-24 10:00:26,561-1-1,0.0000000000
825,2022-09-19 23:54:11,561-1-1,0.1200000000
826,2022-09-20 19:14:12,561-1-1,0.1200000000
827,2022-09-20 20:14:12,561-1-1,0.1200000000
...,...,...,...
52380804,2024-06-24 09:36:59,563-24-2,0.1100000000
52380805,2024-06-24 09:46:59,563-24-2,0.1100000000
52380806,2024-06-24 09:56:59,563-24-2,0.1100000000
52380807,2024-06-24 10:06:59,563-24-2,0.1100000000


##### Case 02. "METER_DATE" Cleansing이 완료된 경우

In [93]:
str_fileCleansed = str('KIER_RAW_' + str_domain + '_Cleansed.csv')
df_raw = pd.read_csv(str_dir_raw + str_fileCleansed, index_col = 0)
print(df_raw.shape, ' /// ', df_raw.columns)
df_raw

(52379163, 3)  ///  Index(['METER_DATE', 'HOUSE_ID', 'GAS_ACCU_FLOW'], dtype='object')


,METER_DATE,HOUSE_ID,GAS_ACCU_FLOW
823,2022-08-10 15:30:09,561-1-1,0.0000000000
824,2022-08-24 10:00:26,561-1-1,0.0000000000
825,2022-09-19 23:54:11,561-1-1,0.1200000000
826,2022-09-20 19:14:12,561-1-1,0.1200000000
827,2022-09-20 20:14:12,561-1-1,0.1200000000
...,...,...,...
52380804,2024-06-24 09:36:59,563-24-2,0.1100000000
52380805,2024-06-24 09:46:59,563-24-2,0.1100000000
52380806,2024-06-24 09:56:59,563-24-2,0.1100000000
52380807,2024-06-24 10:06:59,563-24-2,0.1100000000


#### 02-02. Data Separation
각 세대별 적산값을 파일로 저장

In [94]:
## ▶ 각 세대별 적산값을 Column으로 분리 및 각 CSV로 저장
dt_start, dt_end = 0, 0

list_HID = df_kier_hList['HOUSE_ID'].drop_duplicates()
for house in list_HID:
    print(str(house) + " H")
    str_col_accu_h = str_col_accu + "_" + house

    df_h_tmp = df_raw[(df_raw['HOUSE_ID'] == house)].reset_index()
    df_h_tmp['METER_DATE'] = pd.to_datetime(df_h_tmp['METER_DATE'])
    df_h_tmp = com_date.create_col_ymdhm(df_h_tmp, 'METER_DATE')

    ## METER_DATE의 Minute이 10분 단위가 아닌 경우, 10분 단위로 변경
    for i in range(0, len(df_h_tmp)) : df_h_tmp['MINUTE'].iloc[i] = (math.floor(df_h_tmp['MINUTE'].iloc[i]/10))*10
    
    int_len_start = len(df_h_tmp)
    ## NaN값에 대한 Interpolation
    ## 1) 변경된 시간값을 기반으로 "METER_DATE" Column을 다시 생성 
    ## 2) 생성된 "METER_DATE" Column을 기반으로 중복 제거 (시간 중첩으로 인한 MemoryError 방지)
    ## 3) 열 정리 : ['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu]
    ## 4) 적산 Nan값을 제거
    ## 5) 사용량 Column을 str_col_accu_h로 변경 (추후 취합 목적)
    df_h_tmp = com_date.create_col_datetime(df_h_tmp, 'METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE')
    df_h_tmp = df_h_tmp.drop_duplicates(subset = 'METER_DATE')[['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu]].dropna().rename(columns = {str_col_accu : str_col_accu_h})

    ## ▶ 최적의 Period 계산 및 dt_start, dt_end 변수에 저장
    if dt_start == 0 : dt_start = df_h_tmp['METER_DATE'].min()
    else : 
        if dt_start < df_h_tmp['METER_DATE'].min(): dt_start = df_h_tmp['METER_DATE'].min()
    
    if dt_end == 0 : dt_end = df_h_tmp['METER_DATE'].max()
    else : 
        if dt_start > df_h_tmp['METER_DATE'].max() : dt_start = df_h_tmp['METER_DATE'].max()

    
    ## 만약 작업 전후로 Data Shape이 변동되었다면, 변동사항 출력
    int_len_end = len(df_h_tmp)
    if int_len_start != int_len_end: print(int_len_start, ' / ', int_len_end)

    ## 해당 Dataframe을 각 CSV로 Export
    str_df_h = str('KIER_' + str_domain + '_' + str(house) + '_ACCU_01_Raw.csv')
    df_h_tmp.to_csv(str_dirName_h + str_df_h)

print(dt_start, ' / ',dt_end)

561-1-1 H
179956  /  89906
561-1-2 H
173802  /  85459
561-1-3 H
179750  /  87827
561-1-4 H
167066  /  81921
561-2-1 H
179754  /  88401
561-2-2 H
171822  /  85506
561-2-3 H
173418  /  85800
561-2-4 H
179908  /  89936
561-3-1 H
167302  /  83253
561-3-2 H
173964  /  86909
561-3-3 H
170826  /  85374
561-3-4 H
170872  /  85340
561-4-1 H
176260  /  88119
561-4-2 H
179968  /  89938
561-4-3 H
175796  /  87684
561-4-4 H
171328  /  85551
561-5-1 H
173718  /  85370
561-5-2 H
172786  /  85377
561-5-3 H
173820  /  84436
561-5-4 H
179958  /  89881
561-6-1 H
174320  /  86263
561-6-2 H
159192  /  79190
561-6-3 H
179404  /  89667
561-6-4 H
171038  /  85483
561-7-1 H
173920  /  86895
561-7-2 H
168294  /  83866
561-7-3 H
169052  /  83744
561-7-4 H
171664  /  85630
561-8-1 H
170742  /  84830
561-8-2 H
178748  /  88547
561-8-3 H
180014  /  89810
561-8-4 H
162160  /  81002
561-9-1 H
179418  /  88567
561-9-2 H
173158  /  86383
561-9-3 H
172200  /  86031
561-9-4 H
168598  /  84226
561-10-1 H
172600  /  86247


In [95]:
## 계산된 Period에 대한 Date Dataframe 생성
df_kier_h_Combined = com_date.create_df_dt(pd.DataFrame(), 'METER_DATE', dt_start, dt_end, '10min')[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']]

## METER_DATE의 Minute이 10분 단위가 아닌 경우, 10분 단위로 변경
for i in range(0, len(df_kier_h_Combined)) : df_kier_h_Combined['MINUTE'].iloc[i] = (math.floor(df_kier_h_Combined['MINUTE'].iloc[i]/10))*10
df_kier_h_Combined

,YEAR,MONTH,DAY,HOUR,MINUTE
0,2022,8,26,20,50
1,2022,8,26,21,0
2,2022,8,26,21,10
3,2022,8,26,21,20
4,2022,8,26,21,30
...,...,...,...,...,...
96124,2024,6,24,9,30
96125,2024,6,24,9,40
96126,2024,6,24,9,50
96127,2024,6,24,10,0


### 03. 적산 사용량
모든 호실의 사용량을 변수화하여 한 데이터셋에 Combine

In [96]:
list_HID = df_kier_hList['HOUSE_ID'].drop_duplicates()
for house in list_HID:
    print(str(house) + " H")

    str_col_accu_h = str_col_accu + "_" + house
    str_file = 'KIER_' + str_domain + '_' + str(house) + '_ACCU_01_Raw.csv'
    df_h_tmp = pd.read_csv(str_dirName_h + str_file, index_col = 0).reset_index()[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu_h]]

    df_kier_h_Combined = pd.merge(df_kier_h_Combined, df_h_tmp, how = 'left', on = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'])
    print(df_kier_h_Combined.shape, ' /// ', df_kier_h_Combined.columns)

str_file = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
df_kier_h_Combined.to_csv(str_dirName_h + str_file)
print(df_kier_h_Combined.shape, ' /// ', df_kier_h_Combined.columns)
df_kier_h_Combined

561-1-1 H
(96129, 6)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'GAS_ACCU_FLOW_561-1-1'], dtype='object')
561-1-2 H
(96129, 7)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'GAS_ACCU_FLOW_561-1-1',
       'GAS_ACCU_FLOW_561-1-2'],
      dtype='object')
561-1-3 H
(96129, 8)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'GAS_ACCU_FLOW_561-1-1',
       'GAS_ACCU_FLOW_561-1-2', 'GAS_ACCU_FLOW_561-1-3'],
      dtype='object')
561-1-4 H
(96129, 9)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'GAS_ACCU_FLOW_561-1-1',
       'GAS_ACCU_FLOW_561-1-2', 'GAS_ACCU_FLOW_561-1-3',
       'GAS_ACCU_FLOW_561-1-4'],
      dtype='object')
561-2-1 H
(96129, 10)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'GAS_ACCU_FLOW_561-1-1',
       'GAS_ACCU_FLOW_561-1-2', 'GAS_ACCU_FLOW_561-1-3',
       'GAS_ACCU_FLOW_561-1-4', 'GAS_ACCU_FLOW_561-2-1'],
      dtype='object')
561-2-2 H
(96129, 11)  ///  Index(['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'GAS_ACCU_FLOW_5

,YEAR,MONTH,DAY,HOUR,MINUTE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,...,GAS_ACCU_FLOW_563-22-3,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2
0,2022,8,26,20,50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000000000,NaN,NaN,NaN
1,2022,8,26,21,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,8,26,21,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,8,26,21,20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,8,26,21,30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96124,2024,6,24,9,30,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96125,2024,6,24,9,40,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96126,2024,6,24,9,50,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
96127,2024,6,24,10,0,0.1300000000,0.1300000000,116.2500000000,NaN,0.1100000000,...,107.1100000000,80.5600000000,98.2200000000,31.0500000000,0.1000000000,0.1200000000,119.8000000000,123.3100000000,159.0500000000,0.1100000000
